In [238]:
import sqlite3
import pandas as pd
import os
import copy
import regex as re
pd.options.mode.chained_assignment = None

In [239]:
process_path = os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data","02_processed","intermediate.db")
print(process_path)

/home/jupyter/Team-Prophecy/Data/02_processed/intermediate.db


In [240]:
output_path = os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data","03_output_for_tableau")
print(output_path)

/home/jupyter/Team-Prophecy/Data/03_output_for_tableau


In [241]:
process_connection = sqlite3.connect(process_path)

In [242]:
inputs = {    
    'prog_desc'     : "All",
    'courses'       : "All",
    'mod'           : "F2F", #F2F
    'visa'          : "F1 Visa", #F1 Visa
    'required_only' : True,
    'ExpN_eat'      : 150
}

In [243]:
# Building the model parameters
model_params = {
    'prog_desc'         : inputs['prog_desc'],
    'courses'           : inputs['courses'],
    'mod'               : inputs['mod'],
    'visa'              : inputs['visa'],
    'ExpN_eat'          : inputs['ExpN_eat'],
    'enr_hist'          : 4 if inputs['visa'] == 'F1 Visa' else 8,
    'filters'           : {
        'prog_filter'   : inputs['prog_desc'] != 'All',
        'course_filter' : inputs['courses'] != 'All',
        'visa_filter'   : inputs['visa'] != 'combined',
        'required_only' : inputs['required_only']
    },
    'groupby'           : {
        'mod_groupby'   : inputs['mod'] != 'combined'
    }
}

In [244]:
def list_to_str(l):
    """
    Create a string in the format of "('a', 'b', 'c', 'd')" from list ['a', 'b', 'c', 'd']
    """
    return str(tuple(l))

In [245]:
student_stat_query = """
    SELECT rs.reg_term_code, rs.reg_stu_id, crs, rs.sect_id, rs.reg_new_ret_stu, rs.reg_final_status
    FROM registration_status rs
    WHERE 1=1 
"""
student_details_query = """
    SELECT stu_admit_term_code, stu_college, stu_deg_level, stu_dept, stu_id, stu_res, stu_prog, stu_visa, stu_bam
    FROM student_details 
    WHERE 1=1 
"""

program_course_offerings = """
    SELECT * FROM PROGRAM_COURSE_OFFERINGS 
"""

# Program Filter
if model_params['filters']['prog_filter']:
    student_details_query += f"AND stu_prog = '{model_params['prog_desc']}' "

# Course Filter
if model_params['filters']['course_filter']:
    student_stat_query += f"AND crs IN {list_to_str(model_params['courses'])} "

# Visa Filter
if model_params['filters']['visa_filter']:
    student_details_query += f"AND stu_visa = '{model_params['visa']}' "

In [246]:
student_stat = pd.DataFrame(process_connection.execute(student_stat_query).fetchall(),columns=["reg_term_code", "student_id", "crs", "sect_id", "returning_student", "reg_status"])

#WHERE reg_final_status IN ('W','R')
#GROUP BY reg_term_code, crs, sect_id, reg_final_status

student_details = pd.DataFrame(process_connection.execute(student_details_query).fetchall(),columns=["reg_term_code","stu_college","stu_deg_level","stu_dept","student_id",
                                                                                                     "stu_res","stu_prog","stu_visa","stu_bam"])

pco_df = pd.DataFrame(process_connection.execute(program_course_offerings),columns=["stu_prog","crs","required"])

total_stat = student_stat.merge(student_details, on=["reg_term_code","student_id"], how="inner").fillna(0)
total_stat = total_stat.merge(pco_df, on=["stu_prog","crs"])

In [247]:
required_dictionary = pco_df.drop_duplicates().set_index(["stu_prog","crs"])["required"].to_dict()
required_dictionary = {f"({k[0]},{k[1]})":v for k,v in required_dictionary.items()}

In [248]:
if model_params['filters']['required_only']:
    total_stat = total_stat.loc[total_stat["required"] == 1,:]

In [249]:
total_student_population = total_stat[["reg_term_code"]].groupby("reg_term_code").count().reset_index()

In [250]:
reg_term_values = dict(total_stat.groupby(["reg_term_code"])["reg_term_code"].count())

In [251]:
#program_limit_values = dict(total_stat.groupby(["stu_prog"])["stu_prog"].count())

In [252]:
req_courses_context = total_stat.loc[total_stat["reg_status"] == "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()
#nonregistered_courses = total_courses.loc[total_courses["reg_status"] != "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()

Keep in mind: Our registered values will be dependent on what's in our subset.

In [253]:
#req_courses_context = registered_courses.merge(ref_course[["reg_term_code", "crs","sect_id", "cum_total_enrollment"]], on=["reg_term_code", "crs","sect_id"], how="inner").drop_duplicates()
#req_courses_context.loc[req_courses_context["cum_total_enrollment"] < req_courses_context["reg_status"],["cum_total_enrollment"]]  = req_courses_context["reg_status"]
#req_courses_context.columns

In [254]:
reg_term_courses_context = req_courses_context.groupby(["reg_term_code"])

In [255]:
from ortools.linear_solver import pywraplp
from itertools import chain

What we're going to do
-> One thing will be a script file that runs automatically behind the scenes
-> In the frontend, what we're going to do is define an input dictionary
    This is the code that we want: Only international students, and we'll have 

In [256]:
all_semesters = list(reg_term_values.keys())
#program_limit_values <- included because we need a way to constraint students
all_pcs_list = list(req_courses_context[["stu_prog","crs","sect_id"]].drop_duplicates().itertuples(index=False,name=None))
all_pcs_dict = {}
#References what program contains which class
pc_dict = {}
cs_dict = {}
for t_pcs in all_pcs_list:
    if t_pcs[0] not in all_pcs_dict:
        pc_dict[t_pcs[0]] = []
        all_pcs_dict[t_pcs[0]] = {}
    pc_dict[t_pcs[0]].append(t_pcs[1])
    if t_pcs[1] not in all_pcs_dict[t_pcs[0]]:
        cs_dict[t_pcs[1]] = []
        all_pcs_dict[t_pcs[0]][t_pcs[1]] = []
    cs_dict[t_pcs[1]].append(t_pcs[2])
    all_pcs_dict[t_pcs[0]][t_pcs[1]].append(t_pcs[2])

In [257]:
#req_courses_context
#req_courses_context[["reg_term_code","stu_prog","crs","sect_id","reg_status"]]

In [258]:
req_courses_context["stu_prog"].drop_duplicates().tolist()

['EC-MS-ELEN',
 'EC-MS-ISA',
 'EC-MS-AIT',
 'EC-MS-CEIE',
 'EC-MS-CS',
 'EC-MS-DAEN',
 'EC-MS-ISYS',
 'EC-MS-TCOM',
 'EC-MS-OPRS',
 'EC-MS-STAT',
 'EC-MS-SWE',
 'EC-MS-CPE',
 'EC-MS-SYST',
 'EC-MS-BIOE',
 'EC-MS-DFOR',
 'EC-MS-BSTA']

In [259]:
#Constraints
#total_student_population
#program_limit_values

Fall 2020
150 -> 80 Program_1, 40 Program_2, 30 Program_3
50
1 student -> 15 credits = 3-4 classes

In [260]:
try:
    process_connection.execute("DROP TABLE prior_class_table")
    process_connection.commit()
except:
    print()

try:
    process_connection.execute("DROP TABLE student_results_table")
    process_connection.commit()
except:
    print()
    
try:
    process_connection.execute("DROP TABLE program_results_table")
    process_connection.commit()
except:
    print()

try:
    process_connection.execute("DROP TABLE results_table")
    process_connection.commit()
except:
    print()

In [261]:
process_connection.execute("""
                    CREATE TABLE prior_class_table(
                       student_id INTEGER NOT NULL DEFAULT 0, 
                       course_code TEXT NOT NULL,
                       PRIMARY KEY(student_id, course_code)
                    );
                    """)
process_connection.commit()

In [262]:
process_connection.execute("""
                    CREATE TABLE student_results_table(
                       rec_id INTEGER PRIMARY KEY AUTOINCREMENT DEFAULT 0, 
                       semester TEXT NOT NULL,
                       program TEXT NOT NULL,
                       course_code TEXT NOT NULL,
                       student_id INTEGER NOT NULL,
                       international TEXT NOT NULL DEFAULT 'F',
                       waitlisted TEXT NOT NULL DEFAULT 'F',
                       core_course TEXT NOT NULL DEFAULT 'F' 
                    );
                       """)
process_connection.commit()

In [263]:
process_connection.execute("""
                    CREATE TABLE program_results_table(
                       rec_id INTEGER PRIMARY KEY AUTOINCREMENT DEFAULT 0, 
                       semester TEXT NOT NULL,
                       program TEXT NOT NULL,
                       course_code TEXT NOT NULL,
                       waitlisted TEXT NOT NULL DEFAULT 'F',
                       number_of_students TEXT NOT NULL
                    );
                       """)
process_connection.commit()

In [264]:
process_connection.execute("""
                    CREATE TABLE results_table(
                       rec_id INTEGER PRIMARY KEY AUTOINCREMENT DEFAULT 0, 
                       semester TEXT NOT NULL,
                       program TEXT NOT NULL,
                       min_waitlisted_students INTEGER NOT NULL
                    );
                       """)
process_connection.commit()

In [265]:
req_mean = req_courses_context.groupby(["crs","sect_id"]).mean("reg_status").reset_index()
req_med = req_courses_context.groupby(["crs","sect_id"]).median("reg_status").reset_index()
req_mean = req_mean.rename(columns={"reg_status": "mean"})
req_med = req_med.rename(columns={"reg_status": "median"})
req_avg = req_mean.merge(req_med,on=["crs","sect_id"],how="inner")
req_avg["avg"] = round(req_avg[["mean","median"]].max(axis=1))
req_avg = req_avg[["crs","sect_id","avg"]]

In [266]:
course_sect_avg_dict = req_avg.set_index(["crs","sect_id"])["avg"].to_dict()

In [267]:
req_mean = req_courses_context.groupby(["crs"]).mean("reg_status").reset_index()
req_med = req_courses_context.groupby(["crs"]).median("reg_status").reset_index()
req_mean = req_mean.rename(columns={"reg_status": "mean"})
req_med = req_med.rename(columns={"reg_status": "median"})
req_avg = req_mean.merge(req_med,on=["crs"],how="inner")
req_avg["avg"] = round(req_avg[["mean","median"]].max(axis=1))
req_avg = req_avg[["crs","avg"]]

In [268]:
course_avg_dict = req_avg.set_index(["crs"])["avg"].to_dict()

In [269]:
def modelSolve(a_sem,cxt_grp,prior_student_info):
    #WE HAVE RATIOS HERE IN CASE WE NEED TO DO MORE ANALYSIS OR MATCH IT TO 1
    #
    #These will be our constraints
    CAP_s = model_params['ExpN_eat']
    
    cxt_grp["reg_status"] = cxt_grp["reg_status"]/reg_term_values[a_sem]
    #total_students_list = cxt_grp["cum_total_enrollment"]/reg_term_values[a_sem]
    pcs_weighting_ratio = cxt_grp[["reg_term_code","stu_prog","crs","sect_id","reg_status"]].set_index(["crs"])["reg_status"].to_dict()
    pcs_weighting_ratio = {k : v for k,v in pcs_weighting_ratio.items()} #"("+",".join(k)+")"
    #Now that we have the ratios, we can begin with the objective function.
    #
    program_ref = {}
    course_ref = {}
    
    program_w_ref = {}
    course_w_ref = {}
    
    course_student_ref = {}
    course_student_w_ref = {}
    
    #Total Decision and Constant Variables Used
    decision_vars = {}
    const_vars = {}
    
    #Student ID Decision Variables
    x_decision_vars = {}
    w_decision_vars = {}
    a_decision_vars = {}
    b_decision_vars = {}
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("Cannot get solver")
        return
        
    cs_l = cxt_grp["crs"].drop_duplicates().tolist()

    #
    s_inf = solver.infinity()
    
    all_cs = dict(cxt_grp[["crs","sect_id"]].drop_duplicates().itertuples(index=False,name=None))
    
    for cs in all_cs.keys():
        if cs not in course_ref.keys():
            course_ref[cs] = []
            course_w_ref[cs] = []
            
        if cs not in course_student_ref.keys():
            course_student_ref[cs] = {}
            course_student_w_ref[cs] = {}
        
        for student_id in range(1,CAP_s):
            if student_id not in course_student_ref[cs]:
                course_student_ref[cs][student_id] = []
            
            if student_id not in course_student_w_ref[cs]:
                course_student_w_ref[cs][student_id] = []
            #######################################################
            #DEFINES DECISION VARS
            # -> Establishes all student id based on the CAP_s provided
            #######################################################
            for sect in all_cs[cs]:
                #We'll have two primary decision variables: Waitlist and Student
                #This will be a minimization function for our purposes, as we want all students
                # to be included.

                x_name = f"x({cs},{sect},{student_id})"
                w_name = f"w({cs},{sect},{student_id})"

                decision_vars[x_name] = solver.IntVar(0,1,x_name)
                decision_vars[w_name] = solver.IntVar(0,1,w_name)

                if student_id not in x_decision_vars:
                    x_decision_vars[student_id] = []
                if student_id not in w_decision_vars:
                    w_decision_vars[student_id] = []

                x_decision_vars[student_id].append(decision_vars[x_name])
                w_decision_vars[student_id].append(decision_vars[w_name])

                course_ref[cs].append(decision_vars[x_name])
                course_w_ref[cs].append(decision_vars[w_name])

                course_student_ref[cs][student_id].append(decision_vars[x_name])
                course_student_w_ref[cs][student_id].append(decision_vars[w_name])
    
        # This addresses all values associated with courses. Sections will come later, but for now let's establish a baseline.
        const_vars[f"01_course_with_w_{cs}_constr_ratio"] = solver.Add(pcs_weighting_ratio[cs]*solver.Sum(course_ref[cs]) <= course_avg_dict[cs] + solver.Sum(course_w_ref[cs]), name=f"01_course_with_w_{cs}_constr_ratio")
        const_vars[f"02_course_with_w_{cs}_constr"] = solver.Add(solver.Sum(course_ref[cs]) <= course_avg_dict[cs], name=f"02_course_with_w_{cs}_constr")
        for student_id in range(1,CAP_s): 
            const_vars[f"03_course_{cs}_sections_waitlist_constr"] = solver.Add(solver.Sum(course_student_w_ref[cs][student_id]) <= 1, name=f"03_course_{cs}_sections_waitlist_constr")
            const_vars[f"04_course_{cs}_sections_constr"] = solver.Add(solver.Sum(course_student_ref[cs][student_id]) <= 1, name=f"04_course_{cs}_sections_constr")

            const_vars[f"05_one_regstat_{student_id}"] = solver.Add(solver.Sum(course_student_ref[cs][student_id]) + solver.Sum(course_student_w_ref[cs][student_id]) == 1, name=f"05_one_regstat_{student_id}")

            a_name = f"a({cs},{a_sem})"
            b_name = f"b({cs},{a_sem})"
            a_decision_vars[a_name] = solver.IntVar(0,s_inf,a_name)
            b_decision_vars[b_name] = solver.IntVar(0,s_inf,b_name)

            const_vars[f"06_student_max_classes_bounds_{student_id}"] = solver.Add(solver.Sum(x_decision_vars[student_id]) <= 3, name=f"06_student_max_classes_bounds_{student_id}")
            #const_vars[f"07_student_min_classes_bounds_{student_id}"] = solver.Add(solver.Sum(x_decision_vars[student_id]) >= 1, name=f"07_student_min_classes_bounds_{student_id}")
        const_vars[f"07_organization_course_{cs}_const"] = solver.Add(solver.Sum([solver.Sum(course_student_ref[cs][student_id]) for student_id in range(1,CAP_s)]) == a_decision_vars[a_name],name=f"07_organization_course_{cs}_const")
        const_vars[f"08_organization_wait_course_{cs}_const"] = solver.Add(solver.Sum([solver.Sum(course_student_w_ref[cs][student_id]) for student_id in range(1,CAP_s)]) == b_decision_vars[b_name],name=f"08_organization_wait_course_{cs}_const")

    
    # Minimization function 
    # -> For later * a_decision_vars[student_id]
    
    solver.Minimize(solver.Sum([solver.Sum(w_decision_vars[student_id])
                                for student_id in range(1,CAP_s)]))
    solver.Solve()
    
    return solver, x_decision_vars, w_decision_vars, a_decision_vars, b_decision_vars

In [270]:
#passed_student_info = {}
student_results_table = []
program_results_table = []
results_table = []
for a_sem in all_semesters:
    cxt_grp = reg_term_courses_context.get_group(a_sem)
    prg_list = cxt_grp["stu_prog"].drop_duplicates().tolist()
    for p in prg_list:
        solver, x_decision_vars, w_decision_vars, \
            a_decision_vars,b_decision_vars = modelSolve(a_sem,cxt_grp.loc[cxt_grp["stu_prog"] == p,:],prior_student_info)
        #print('Objective value =', solver.Objective().Value())
        #print(f"Registered Students under the Classes taken for Program {p} on {a_sem}:")
        
        for i in list(a_decision_vars.values()):
            #print(f"{i.name()} -> {i.SolutionValue()}")
            course_res = re.findall(r"\(([A-Z0-9]+),",i.name())[0]
            program_results_table.append((a_sem,p,course_res,"F",i.SolutionValue()))
        #print(f"Waitlisted Students under the Classes taken for Program {p} on {a_sem}:")
        for i in list(b_decision_vars.values()):
            #print(f"{i.name()} -> {i.SolutionValue()}")
            course_res = re.findall(r"\(([A-Z0-9]+),",i.name())[0]
            program_results_table.append((a_sem,p,course_res,"T",i.SolutionValue()))
        
        #print(f"Retrieving individual students under a given class for Program {p} on {a_sem}:")
        all_students = list(x_decision_vars.keys())
        
        #NOTE: Value for international students set to true because we haven't gotten to that point yet
        for i in all_students:
            rdec = range(0,len(x_decision_vars[i]))
            for k in rdec:
                solVal = x_decision_vars[i][k].SolutionValue()
                if solVal == 0:
                    continue
                retained_val = re.findall(r"\(([A-Z0-9]+),([0-9]*),([0-9]*)",x_decision_vars[i][k].name())
                #print(retained_val)
                for r in retained_val:
                    course = r[0]
                    sect = r[1]
                    stu_id = r[2]
                    req_val = "T" if required_dictionary[f"({p},{course})"] == 1 else "F"

                    student_results_table.append((a_sem,p,course,stu_id,"T","F",req_val))
                #print(f"Course: {course} - Required {req_val}, Sect: {sect}, Stu_ID: {stu_id} -> {solVal}")
            
        #print(f"Retrieving waitlisted individual students under a given class for Program {p} on {a_sem}:")
        for i in all_students:
            rdec = range(0,len(w_decision_vars[i]))
            for k in rdec:
                solVal = w_decision_vars[i][k].SolutionValue()
                if solVal == 0:
                    continue
                retained_val = re.findall(r"\(([A-Z0-9]+),([0-9]*),([0-9]*)",w_decision_vars[i][k].name())
                for r in retained_val:
                    course = r[0]
                    sect = r[1]
                    stu_id = r[2]
                    req_val = "T" if required_dictionary[f"({p},{course})"] == 1 else "F"

                    student_results_table.append((a_sem,p,course,stu_id,"T","T",req_val))
                #print(f"{i.name()} -> {i.SolutionValue()}")
        
        results_table.append((a_sem,p,solver.Objective().Value()))
        #break
    #process_connection.executemany("INSERT INTO prior_class_table(student_id, course_code)")
    #process_connection.commit()

    process_connection.executemany("INSERT INTO results_table(semester, program, min_waitlisted_students) VALUES(?,?,?)",
                                  results_table)
    process_connection.commit()
    
    process_connection.executemany("INSERT INTO program_results_table(semester, program, course_code, waitlisted, number_of_students) VALUES(?,?,?,?,?)",
                                  program_results_table)
    process_connection.commit()
    
    process_connection.executemany("INSERT INTO student_results_table(semester, program, course_code, student_id, international, waitlisted, core_course) VALUES(?,?,?,?,?,?,?)",
                                  student_results_table)
    process_connection.commit()
    #break
#

In [271]:
student_results_table = pd.DataFrame(process_connection.execute("SELECT * FROM student_results_table").fetchall(),columns=["rec_id","semester","program","course_code","student_id","international","waitlisted","core_course"])
program_results_table = pd.DataFrame(process_connection.execute("SELECT * FROM program_results_table").fetchall(),columns=["rec_id","semester","program","course_code","waitlisted","number_of_students"])
results_table = pd.DataFrame(process_connection.execute("SELECT * FROM results_table").fetchall(),columns=["rec_id","semester","program","min_waitlisted_students"])

In [272]:
output_path

'/home/jupyter/Team-Prophecy/Data/03_output_for_tableau'

In [273]:
student_results_table.drop("rec_id",axis=1).to_csv(output_path+os.sep+"student_results.csv")
program_results_table.drop("rec_id",axis=1).to_csv(output_path+os.sep+"program_results.csv")
results_table.drop("rec_id",axis=1).to_csv(output_path+os.sep+"overall_results.csv")

**NOW WE CAN GET INTO THE INTERESTING STUFF**

This is for when I have more data; only focusing on what I know for certain given the total number of students available.